In [1]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import mplfinance as mpf

symbol = 'ET'  # Change to desired stock ticker
start_date = '2024-01-01'
# end_date = '2025-02-01'

stock_data = yf.download(symbol, start=start_date)
stock_data = stock_data.xs(symbol, axis=1, level='Ticker').reset_index()
stock_data['100_MA'] = stock_data['Close'].rolling(window=100).mean()

delta = stock_data['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
stock_data['RSI'] = 100 - (100 / (1 + rs))

stock_data['Middle_Band'] = stock_data['Close'].rolling(window=20).mean()
stock_data['Upper_Band'] = stock_data['Middle_Band'] + 2 * stock_data['Close'].rolling(window=20).std()
stock_data['Lower_Band'] = stock_data['Middle_Band'] - 2 * stock_data['Close'].rolling(window=20).std()
stock_data = stock_data.query('Date >"2024-06-01"')


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [2]:
stock_data.tail(7)

Price,Date,Close,High,Low,Open,Volume,100_MA,RSI,Middle_Band,Upper_Band,Lower_Band
265,2025-01-23,20.723989,20.930640,20.645266,20.743671,13095500,17.238201,79.051379,19.524440,20.653630,18.395250
266,2025-01-24,20.714149,20.901119,20.487820,20.891278,20462400,17.291203,77.685972,19.623336,20.807006,18.439667
267,2025-01-27,19.493935,20.094203,19.208563,20.054840,46862500,17.330650,52.367691,19.633669,20.808562,18.458776
268,2025-01-28,20.025320,20.064681,19.484094,19.730105,21954000,17.376088,58.880782,19.687299,20.828761,18.545838
269,2025-01-29,20.202448,20.320532,20.084362,20.143404,9429900,17.424358,62.500029,19.748802,20.860233,18.637372
270,2025-01-30,20.743671,20.812554,20.310691,20.310691,26849200,17.477076,65.100682,19.823098,20.993416,18.652779
271,2025-01-31,20.153244,20.822395,20.113883,20.812554,17431300,17.525144,61.134453,19.866888,21.016599,18.717176


In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['100_MA'], mode='lines', name='100-Day MA', line=dict(color='red')))
fig.update_layout(title=f'{symbol} Daily Stock Price with 100-Day Moving Average', xaxis_title='Date', yaxis_title='Price', width=800, height=500)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['RSI'], mode='lines', name='RSI'))
fig.add_hline(y=70, line=dict(color='red', dash='dash'))
fig.add_hline(y=30, line=dict(color='green', dash='dash'))
fig.update_layout(title=f'{symbol} Daily Stock Price with 100-Day Moving Average', xaxis_title='Date', yaxis_title='Price', width=800, height=500)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Upper_Band'], mode='lines', name='Upper Band', line=dict(color='red')))
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Middle_Band'], mode='lines', name='Middle Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Lower_Band'], mode='lines', name='Lower Band', line=dict(color='green')))
fig.update_layout(title=f'{symbol} Daily Stock Price with 100-Day Moving Average', xaxis_title='Date', yaxis_title='Price', width=800, height=500)
fig.show()

stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.set_index('Date', inplace=True)
mpf.plot(stock_data, type='candle', volume=True, style='charles', title=f'{symbol} Candlestick Chart', figsize=(8,5))

